# The Long Goodbye

## EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig,ax =  plt.subplots(1,2,figsize=(12,9))
sns.kdeplot(train.x1,label='train_x1',ax=ax[0,0])
sns.kdeplot(test.x1,label='test_x1',ax=ax[0,0])
ax[0,0].set_title('x1')
sns.kdeplot(train.x2,label='train_x2',ax=ax[0,1])
sns.kdeplot(test.x2,label='test_x2',ax=ax[0,1])
ax[0,1].set_title('x2')

In [ ]:
fig,ax =  plt.subplots(1,2,figsize=(12,9))
sns.scatterplot(train.x1, train.y_, ax=ax[0,0])
sns.scatterplot(train.x2, train.y_, ax=ax[0,1])

In [ ]:
sns.heatmap(df.T.isna(), cmap='Blues')
ax.set_title('Fields with Missing Values', fontsize=16) #看特征同时为null的情况
ax.set_ylabel('Year', fontsize=16)
ax.set_xlabel('Feature', fontsize=16)

In [ ]:
sns.pairplot(df[['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']],corner=True);

In [ ]:
#可能会发现新世界
plt.scatter(x,y) 
alpha=0.01 #调整透明度
s=1 #标点大小
edgecolor='none'
ax.set_xlim(xmin,xmax)
ax.set_ylim(ymin,ymax)

In [ ]:
#降维并且可视化
from sklearn.manifold import TSNE
tsne = TSNE()
tsne.fit_transform(X) 
tsne = pd.DataFrame(tsne.embedding_) 

In [ ]:
pca=PCA(n_components=1)
newData=pca.fit_transform(X)
pca.n_components
pca.explained_variance_ratio_
pca.explained_variance_

In [ ]:
# 如果KS统计量小或p值高，则分布相同(我们不能拒绝两个样本的分布相同的假设)。
from scipy.stats import ks_2samp

def corr(train, test,cols):
    # assuming first column is `class_name_id`


    for class_name in cols:
        # all correlations
        print('\n Class: %s' % class_name)

        ks_stat, p_value = ks_2samp(train[class_name].values,
                                    test[class_name].values)
        print(' Kolmogorov-Smirnov test:    KS-stat = %.6f    p-value = %.3e\n'
              % (ks_stat, p_value))

corr(train, test,cols=['year','month','day','hour','minute','weekday'])

## Feature Engineering

去掉空值多的(nan值一样的取unique值多的)，方差小的，相关性高的(train_reduced = VarianceThreshold(1.5).fit_transform(train))，特征重要性低的。  
from feature_selector import FeatureSelector  
https://nbviewer.jupyter.org/github/WillKoehrsen/feature-selector/blob/master/Feature%20Selector%20Usage.ipynb



树模型
空值替换为-999和不替换两种方法比较一下
LGB类别变量 label encoding(作为数字)和作为category两种方法比较一下

In [ ]:
# Encoding

# use target encoding to encode two categorical features
enc = TargetEncoder(cols=['CHAS', 'RAD'])

# transform the datasets
training_numeric_dataset = enc.fit_transform(X_train, y_train)
testing_numeric_dataset = enc.transform(X_test)

# encoding function

# frequency encode
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col], df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col + "FE"
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype("float32")
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype("float32")
        print(col)


# label encode
def encode_LE(col, train=X_train, test=X_test, verbose=True):
    df_comb = pd.concat([train[col], test[col]], axis=0)
    df_comb, _ = pd.factorize(df_comb[col])
    nm = col
    if df_comb.max() > 32000:
        train[nm] = df_comb[0: len(train)].astype("float32")
        test[nm] = df_comb[len(train):].astype("float32")
    else:
        train[nm] = df_comb[0: len(train)].astype("float16")
        test[nm] = df_comb[len(train):].astype("float16")
    del df_comb
    gc.collect()
    if verbose:
        print(col)


def encode_AG(main_columns, uids, aggregations=["mean"], df_train=X_train, df_test=X_test, fillna=True, usena=False):
    for main_column in main_columns:
        for col in uids:
            for agg_type in aggregations:
                new_column = main_column + "_" + col + "_" + agg_type
                temp_df = pd.concat([df_train[[col, main_column]], df_test[[col, main_column]]])
                if usena:
                    temp_df.loc[temp_df[main_column] == -1, main_column] = np.nan

                #求每个uid下，该col的均值或标准差
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                    columns={agg_type: new_column})
                #将uid设成index
                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_column].to_dict()
                #temp_df是一个映射字典
                df_train[new_column] = df_train[col].map(temp_df).astype("float32")
                df_test[new_column] = df_test[col].map(temp_df).astype("float32")
                if fillna:
                    df_train[new_column].fillna(-1, inplace=True)
                    df_test[new_column].fillna(-1, inplace=True)
                print(new_column)


# COMBINE FEATURES交叉特征
def encode_CB(col1, col2, df1=X_train, df2=X_test):
    nm = col1 + '_' + col2
    df1[nm] = df1[col1].astype(str) + '_' + df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str) + '_' + df2[col2].astype(str)
    encode_LE(nm, verbose=False)
    print(nm, ', ', end='')


# GROUP AGGREGATION NUNIQUE
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:
        for col in uids:
            comb = pd.concat([train_df[[col] + [main_column]], test_df[[col] + [main_column]]], axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col + '_' + main_column + '_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col + '_' + main_column + '_ct'] = test_df[col].map(mp).astype('float32')
            print(col + '_' + main_column + '_ct, ', end='')

## Model

In [ ]:
#NN
model = Sequential()  
model.add(Dense(32, input_dim=4, init='uniform', activation='relu'))
model.add(Dropout(0.3))  
model.add(Dense(16, init='uniform', activation='relu'))
model.add(Dropout(0.3)) 
model.add(Dense(8, init='uniform', activation='relu')) 
model.add(Dropout(0.3)) 
model.add(Dense(2, init='uniform', activation='softmax')) 
opt = optimizers.Nadam(lr=0.0001)
 
model.compile(loss='binary_crossentropy', optimizer=opt,metrics = ['accuracy']) 
model.fit(X_train, y_train, nb_epoch=3000, batch_size=40,verbose=1, validation_data=(X_test,y_test))#不需要class_weight=cw
predictions = model.predict(X_train)

In [ ]:
#LGB
d_train = lgb.Dataset(train[features], train['y_'])
clf = lgb.train(trial.params,d_train) #after optuna fine tune
all_pred = clf.predict(test[features])

In [ ]:
#XGB
d_train = xgb.DMatrix(train[features], train['y_'])
clf = xgb.train(trial.params, d_train, obj=huber_approx_obj)
all_pred = clf.predict(xgb.DMatrix(test[features]))

In [ ]:
#optuna 
#目标函数(objective)，单次试验(trial)，和研究(study). 其中 objective 负责定义待优化函数并指定参/超参数数范围，trial 对应着 objective 的单次执行，而 study 则负责管理优化，决定优化的方式，总试验的次数、试验结果的记录等功能。

import optuna
def objective(trial):

    # Optuna suggest params
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.10),
        'objective': 'huber',
        'lambda': trial.suggest_uniform('lambda', 0, 1),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0, 1),
        'bagging_freq': 2
    }

    score=[]
    tscv = TimeSeriesSplit()
    for train_index, val_index in tscv.split(train):
        X_train, X_val = train.loc[train_index,features], train.loc[val_index,features]
        y_train, y_val = train.loc[train_index,'y_'], train.loc[val_index,'y_']
        d_train = lgb.Dataset(X_train, y_train)
        d_val = lgb.Dataset(X_val, y_val)
        clf = lgb.train(params,d_train)
        val_pred = clf.predict(X_val)
        score.append(mean_absolute_error(val_pred,y_val))
    return np.mean(score)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=60)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
# 如果不指定 params 也是可以的，optuna 将画出所有的参数之间的关系，在这里两者等价。
optuna.visualization.plot_contour(study, params=['x', 'y'])
optuna.visualization.plot_contour(study)

这里提出一种简单的初始化调参方式：1、使用不同的初始化方法，xavier，glorot，he_uniform等，可以把torch或tf.keras支持的各类初始化方式作为超参数；2、不同的超参数多次进行随机初始化；3、每一个初始化之后的网络都直接进行预测，计算评价指标通过这种简单的方式，可以在相对合理的时间里找到最优的初始化方法。


伪标签 详见知乎：求是汪
选概率为0.99 0.01的 硬标签
也有软标签
回归问题 计算两次模型误差最小的
半监督学习：思想很简单，对于标注好的样本一样用交叉熵作为损失函数，对于没标注的样本，则用Consistency Loss。

拒绝演绎
样本偏差对信贷风控模型的影响？
在头部优质客群上所建立的模型，在全量客群上效果自然会衰减。这解释了用单一机构的Y所建立的KGB模型，在其他机构上可能完全失效的现象。如果你测试过三方数据商提供的评分产品，你就会有更深的体会。
推荐：利用两阶段双评分卡来进行拒绝推断探索。全样本预测通过拒绝score1，通过样本预测是否违约score2。score2=f(score1)。然后得到拒绝样本的score2'。利用拒绝推断重新训练是否违约模型。

每一个fold(5 fold)的test set做预测最后拼起来变成oof，然后metrics(train.y,oof.prediction)。否则就是每个fold自己的metrics了。


2个模型ensemble，确定每个模型的权重w，使w加权的oof的metrics最大  
\>2个Start with the one model that has highest CV score. Next iterate through all your additional models and find the one model that combines with the first model to generate the highest two model ensemble CV score. Then search for the best third model. Repeat until ensemble CV does not increase anymore.  
e.g. model_ensemble_forward_selection.py

frequency encoding很重要，做EDA的时候也可以看，或者直接把它们当特征扔进去

样本极度不平衡时，ROC曲线会失真，要看PR曲线。
类别不平衡：我会比较倾向使用改造函数函数来调接样本不平衡的问题，因为无论是过采样或者欠采样，都会导致样本噪声这个问题，而改造损失函数，如调整不同样本的权重，采用Focal Loss等等函数都能有限减少不平衡的问题。

感觉树模型的时序特征也要平衡(去除趋势)

McNemar test statistic可以用来比较两个模型表现。一堆比较模型表现的统计量

earlystop选最好的round可能过拟合

样本还是要shuffle，不然一个batch一样可能会把模型打偏

In [ ]:
#PSI
def psi_for_continue_var(expected_array, actual_array, bins=10, bucket_type='bins', detail=False, save_file_path=None):
    '''
    ----------------------------------------------------------------------
    功能: 计算连续型变量的群体性稳定性指标（population stability index ,PSI）
    ----------------------------------------------------------------------
    :param expected_array: numpy array of original values，基准组
    :param actual_array: numpy array of new values, same size as expected，比较组
    :param bins: number of percentile ranges to bucket the values into，分箱数, 默认为10
    :param bucket_type: string, 分箱模式，'bins'为等距均分，'quantiles'为按等频分箱
    :param detail: bool, 取值为True时输出psi计算的完整表格, 否则只输出最终的psi值
    :param save_file_path: string, csv文件保存路径. 默认值=None. 只有当detail=Ture时才生效.
    ----------------------------------------------------------------------
    :return psi_value: 
            当detail=False时, 类型float, 输出最终psi计算值;
            当detail=True时, 类型pd.DataFrame, 输出psi计算的完整表格。最终psi计算值 = list(psi_value['psi'])[-1]
    ----------------------------------------------------------------------
    示例：
    >>> psi_for_continue_var(expected_array=df['score'][:400],
                             actual_array=df['score'][401:], 
                             bins=5, bucket_type='bins', detail=0)
    >>> 0.0059132756739701245
    ------------
    >>> psi_for_continue_var(expected_array=df['score'][:400],
                             actual_array=df['score'][401:], 
                             bins=5, bucket_type='bins', detail=1)
    >>>
    	score_range	expecteds	expected(%)	actucalsactucal(%)ac - ex(%)ln(ac/ex)psi	max
    0	[0.021,0.2095]	120.0	30.00	152.0	31.02	1.02	0.033434	0.000341	
    1	(0.2095,0.398]	117.0	29.25	140.0	28.57	-0.68	-0.023522	0.000159	
    2	(0.398,0.5865]	81.0	20.25	94.0	19.18	-1.07	-0.054284	0.000577	<<<<<<<
    3	(0.5865,0.7751]	44.0	11.00	55.0	11.22	0.22	0.019801	0.000045	
    4	(0.7751,0.9636]	38.0	9.50	48.0	9.80	0.30	0.031087	0.000091	
    5	>>> summary	400.0	100.00	489.0	100.00	NaN	NaN	0.001214	<<< result
    ----------------------------------------------------------------------
    知识:
    公式： psi = sum(（实际占比-预期占比）* ln(实际占比/预期占比))
    一般认为psi小于0.1时候变量稳定性很高，0.1-0.25一般，大于0.25变量稳定性差，建议重做。
    相对于变量分布(EDD)而言, psi是一个宏观指标, 无法解释两个分布不一致的原因。但可以通过观察每个分箱的sub_psi来判断。
    ----------------------------------------------------------------------
    '''
    import math
    import numpy as np
    import pandas as pd
    
    expected_array = pd.Series(expected_array).dropna()
    actual_array = pd.Series(actual_array).dropna()
    if isinstance(list(expected_array)[0], str) or isinstance(list(actual_array)[0], str):
        raise Exception("输入数据expected_array只能是数值型, 不能为string类型")
        
    """step1: 确定分箱间隔"""
    def scale_range(input_array, scaled_min, scaled_max):
        '''
        ----------------------------------------------------------------------
        功能: 对input_array线性放缩至[scaled_min, scaled_max]
        ----------------------------------------------------------------------
        :param input_array: numpy array of original values, 需放缩的原始数列
        :param scaled_min: float, 放缩后的最小值
        :param scaled_min: float, 放缩后的最大值
        ----------------------------------------------------------------------
        :return input_array: numpy array of original values, 放缩后的数列
        ----------------------------------------------------------------------
        '''
        input_array += -np.min(input_array) # 此时最小值放缩到0
        if scaled_max == scaled_min:
            raise Exception('放缩后的数列scaled_min = scaled_min, 值为{}, 请检查expected_array数值！'.format(scaled_max))
        scaled_slope = np.max(input_array) * 1.0 / (scaled_max - scaled_min)
        input_array /= scaled_slope
        input_array += scaled_min
        return input_array
    
    # 异常处理，所有取值都相同时, 说明该变量是常量, 返回999999
    if np.min(expected_array) == np.max(expected_array):
        return 999999
    
    breakpoints = np.arange(0, bins + 1) / (bins) * 100 # 等距分箱百分比
    if 'bins' == bucket_type:        # 等距分箱
        breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
    elif 'quantiles' == bucket_type: # 等频分箱
        breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])
 
    """step2: 统计区间内样本占比"""
    def generate_counts(arr, breakpoints):
        '''
        ----------------------------------------------------------------------
        功能: Generates counts for each bucket by using the bucket values 
        ----------------------------------------------------------------------
        :param arr: ndarray of actual values
        :param breakpoints: list of bucket values
        ----------------------------------------------------------------------
        :return cnt_array: counts for elements in each bucket, length of breakpoints array minus one
        :return score_range_array: 分箱区间
        ----------------------------------------------------------------------
        '''
        def count_in_range(arr, low, high, start):
            '''
            ----------------------------------------------------------------------
            功能: 统计给定区间内的样本数(Counts elements in array between low and high values)
            ----------------------------------------------------------------------
            :param arr: ndarray of actual values
            :param low: float, 左边界
            :param high: float, 右边界
            :param start: bool, 取值为Ture时，区间闭合方式[low, high],否则为(low, high]
            ----------------------------------------------------------------------
            :return cnt_in_range: int, 给定区间内的样本数
            ----------------------------------------------------------------------
            '''
            if start:
                cnt_in_range = len(np.where(np.logical_and(arr >= low, arr <= high))[0])
            else:
                cnt_in_range = len(np.where(np.logical_and(arr > low, arr <= high))[0])
            return cnt_in_range
 
        cnt_array = np.zeros(len(breakpoints) - 1)
        score_range_array = [''] * (len(breakpoints) - 1)
        for i in range(1, len(breakpoints)):
            cnt_array[i-1] = count_in_range(arr, breakpoints[i-1], breakpoints[i], i==1)
            if 1 == i:
                score_range_array[i-1] = '[' + str(round(breakpoints[i-1], 4)) + ',' + str(round(breakpoints[i], 4)) + ']'
            else:   
                score_range_array[i-1] = '(' + str(round(breakpoints[i-1], 4)) + ',' + str(round(breakpoints[i], 4)) + ']'
                                                                                
        return (cnt_array, score_range_array)
 
    expected_cnt = generate_counts(expected_array, breakpoints)[0]
    expected_percents = expected_cnt / len(expected_array)
    actual_cnt = generate_counts(actual_array, breakpoints)[0]
    actual_percents = actual_cnt / len(actual_array)
    delta_percents = actual_percents - expected_percents
    score_range_array = generate_counts(expected_array, breakpoints)[1]
                                                                               
    """step3: 区间放缩"""
    def sub_psi(e_perc, a_perc):
        '''
        ----------------------------------------------------------------------
        功能: 计算单个分箱内的psi值。Calculate the actual PSI value from comparing the values.
             Update the actual value to a very small number if equal to zero
        ----------------------------------------------------------------------
        :param e_perc: float, 期望占比
        :param a_perc: float, 实际占比
        ----------------------------------------------------------------------
        :return value: float, 单个分箱内的psi值
        ----------------------------------------------------------------------
        '''
        if a_perc == 0: # 实际占比
            a_perc = 0.001
        if e_perc == 0: # 期望占比
            e_perc = 0.001
        value = (e_perc - a_perc) * np.log(e_perc * 1.0 / a_perc)
        return value
    
    """step4: 得到最终稳定性指标"""
    sub_psi_array = [sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents))]
    if detail:
        psi_value = pd.DataFrame()
        psi_value['score_range'] = score_range_array
        psi_value['expecteds'] = expected_cnt
        psi_value['expected(%)'] = expected_percents * 100
        psi_value['actucals'] = actual_cnt
        psi_value['actucal(%)'] = actual_percents * 100
        psi_value['ac - ex(%)'] = delta_percents * 100
        psi_value['actucal(%)'] = psi_value['actucal(%)'].apply(lambda x: round(x, 2))
        psi_value['ac - ex(%)'] = psi_value['ac - ex(%)'].apply(lambda x: round(x, 2))
        psi_value['ln(ac/ex)'] = psi_value.apply(lambda row: np.log((row['actucal(%)'] + 0.001) \
                                                                  / (row['expected(%)'] + 0.001)), axis=1)
        psi_value['psi'] = sub_psi_array
        flag = lambda x: '<<<<<<<' if x == psi_value.psi.max() else ''
        psi_value['max'] = psi_value.psi.apply(flag)
        psi_value = psi_value.append([{'score_range':'>>> summary', 
                                       'expecteds': sum(expected_cnt),
                                       'expected(%)':100, 
                                       'actucals': sum(actual_cnt),
                                       'actucal(%)':100, 
                                       'ac - ex(%)': np.nan,
                                       'ln(ac/ex)': np.nan,
                                       'psi': np.sum(sub_psi_array),
                                       'max':'<<< result'}], ignore_index=True)
        if save_file_path:
            if not isinstance(save_file_path, str):
                raise Exception('参数save_file_path类型必须是str, 同时注意csv文件后缀!')
            elif not save_file_path.endswith('.csv'):
                raise Exception('参数save_file_path不是csv文件后缀，请检查!')
            psi_value.to_csv(save_file_path, encoding='utf-8', index=1)
    else:
        psi_value = np.sum(sub_psi_array)
 
    return psi_value

In [ ]:
#PurgedGroupTimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            
cv = PurgedGroupTimeSeriesSplit(n_splits = n_splits, group_gap = group_gap)
for i, (train, valid) in enumerate(cv.split(X=X, y=y, groups=group)):
    data[train]
    data[valid]

## Other tools

In [ ]:
import logging

logging.basicConfig(level=logging.WARNING,
                    filename='./log.txt',   
                    filemode='w',  
                    format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')
# use logging
logging.info('this is a loggging info message')
logging.debug('this is a loggging debug message')
logging.warning('this is loggging a warning message')
logging.error('this is an loggging error message')
logging.critical('this is a loggging critical message')

try:
    fun()
except Exception as e:
    logging.error('something wrong',exc_info=True)
logging.info('start')
fun()
logging.info('end')

多进程：multiprocess  
每个进程拷贝自己的变量  
多线程：threading  
变量共享  
并且线程由于GIL并不能执行多核任务。

In [ ]:
def reduce_mem_usage(df,inplace=True):
    if not inplace:
        df=df.copy()
    start_mem=df.memory_usage().sum()/1024**2
    print('Memory usage of dataframe is {:.2f}MB'.format(start_mem))
    for col in df.columns:
        col_type=df[col].dtype
        if col_type!=object:
            c_min=df[col].min()
            c_max=df[col].max()
            if str(col_type)[:3]=='int':
                if c_min>np.iinfo(np.int8).min and c_max<np.iinfo(np.int8).max:
                    df[col]=df[col].astype(np.int8)
                elif c_min>np.iinfo(np.int16).min and c_max<np.iinfo(np.int16).max:
                    df[col]=df[col].astype(np.int16)    
                elif c_min>np.iinfo(np.int32).min and c_max<np.iinfo(np.int32).max:
                    df[col]=df[col].astype(np.int32)
                elif c_min>np.iinfo(np.int64).min and c_max<np.iinfo(np.int64).max:
                    df[col]=df[col].astype(np.int64)                    
            else:
                if c_min>np.finfo(np.float16).min and c_max<np.finfo(np.float16).max:
                    df[col]=df[col].astype(np.float16)                
                elif c_min>np.finfo(np.float32).min and c_max<np.finfo(np.float32).max:
                    df[col]=df[col].astype(np.float32) 
                else:
                    df[col]=df[col].astype(np.float64) 
        else:
            df[col]=df[col].astype('category') 
    end_mem=df.memory_usage().sum()/1024**2
    print('Memory usage after optimization is {:.2f}MB'.format(end_mem))   
    print('Decreased by {:.1f}%'.format(100*(start_mem-end_mem)/start_mem))
    return df

高性能的pandas：eval()和query()  
对于过滤的操作，可以使用query()方法。  
Numexpr程序库可以让你在不为中间过程分配全部内存的前提下，完成元素到元素的复合代数式运算。Pandas的eval()和query()工具就是基于Numexpr实现的。

## Git
- git clone
- git branch --list
- git branch -r 查看远程分支
- git checkout 本地分支
- git status
- git add/ git add --all(这样才会把删除操作加进来)
- git commit -m 'msg'
- git push/ git push origin 远程分支名
- git pull 仅针对所在的分支
    - 防止本地修改被override,1). git stash(保存工作现场),git stash pop(恢复工作现场)
    2). git add, git commit.
    - 若有冲突，解决完冲突后，还需要add，commit
    - 看到有++--说明文件被真正改动
- git merge dev(把dev合并到当前分支)
